In [1]:
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import EarlyStopping

# Load the model
# model = load_model("15_point_plastic_classifier_model8.keras")
# model = load_model("15_point_plastic_classifier_model_30_09.keras")
model = load_model("15_point_plastic_classifier_model_2_10_HvL.keras")
# model = load_model("test_model.keras")



In [ ]:
# # Sample data based on your provided example (modify this as needed)
# sample_data = [
#     [2493065, 4558145, 7638499, 8387631, 8388397, 8389475, 4054559, 2106185, 3340801, 5935779, 8389399, 8388961, 8386511, 6563979, 3097997]
# ]

# # Create a function to generate a representative dataset
# def representative_dataset():
#     for data_point in sample_data:
#         # Reshape and convert to float32 (modify this if your model's input format is different)
#         input_data = tf.constant(data_point, shape=(1, 15), dtype=tf.float32)
#         yield [input_data]

# # Convert the model to TFLite format with quantization
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# converter.optimizations = [tf.lite.Optimize.DEFAULT]
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# converter.inference_input_type = tf.uint8
# converter.inference_output_type = tf.uint8
# converter.representative_dataset = representative_dataset  # Specify the representative dataset

# tflite_model = converter.convert()

# # Save the TensorFlow Lite model
# with open('your_model.tflite', 'wb') as f:
#     f.write(tflite_model)

In [ ]:
# data = pd.read_csv("test_resampled.csv")
# data = pd.read_csv("resampled_dataset8_PVC.csv")
data = pd.read_csv("AllTogetherHvL2.csv")

# Separate features and labels
X = data.drop(columns=["Material_Type"])
y = data["Material_Type"]

# Encode labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into train and test sets (Normal)
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=54)


# Normalize the manual input if needed
X_test_normalized = ((X_test - 0) / (8410000 - 0))
# X_test_normalized = (X_test - X_train.min()) / (X_train.max() - X_train.min())

# Make predictions
predictions = model.predict(X_test_normalized)
predicted_labels = np.argmax(predictions, axis=1)
predicted_plastic_types = label_encoder.inverse_transform(predicted_labels)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_normalized, y_test)

# Make predictions
predictions = model.predict(X_test_normalized)

# Get the top 4 predictions for each sample
top_predictions = np.argsort(predictions, axis=1)[:, -5:]
top_labels = top_predictions[:, ::-1]

# print(predictions)

# Reshape the top_labels array to be 1D
top_labels_1d = top_labels.reshape(-1)

# Inverse transform the labels
top_plastic_types = label_encoder.inverse_transform(top_labels_1d)

# Reshape the top_plastic_types array back to 2D
top_plastic_types = top_plastic_types.reshape(top_labels.shape)

# Get the percentage likelihoods for the top 4 predictions
percentage_likelihoods = np.take_along_axis(predictions, top_predictions, axis=1) * 100

# Display test accuracy and example predictions
print(f"Test Accuracy: {test_accuracy}")
for i in range(len(predicted_labels)):
    num_predictions = min(5, len(np.unique(top_labels[i])))
    if y_test[i] == predicted_labels[i]:
        CorF = "Correct,"
    else:
        CorF = "False,"
    print(f"Sample {i+1}: {CorF} it was {label_encoder.classes_[y_test[i]]}")
    for j in range(num_predictions):
        print(f"  Prediction {j+1}: {top_plastic_types[i][j]} with {percentage_likelihoods[i][1-j]:.2f}% likelihood")



10/10 [==============================] - 0s 2ms/step


10/10 [==============================] - 0s 2ms/step
Test Accuracy: 0.9775640964508057
Sample 1: Correct, it was PET
  Prediction 1: PET with 0.00% likelihood
  Prediction 2: PP with 0.00% likelihood
  Prediction 3: Unknown with 100.00% likelihood
  Prediction 4: HDPE with 0.00% likelihood
  Prediction 5: PVC with 0.00% likelihood
Sample 2: Correct, it was PET
  Prediction 1: PET with 0.00% likelihood
  Prediction 2: Unknown with 0.00% likelihood
  Prediction 3: PVC with 100.00% likelihood
  Prediction 4: HDPE with 0.00% likelihood
  Prediction 5: PP with 0.00% likelihood
Sample 3: Correct, it was Unknown
  Prediction 1: Unknown with 0.01% likelihood
  Prediction 2: HDPE with 0.00% likelihood
  Prediction 3: PP with 95.94% likelihood
  Prediction 4: LDPE with 4.03% likelihood
  Prediction 5: PVC with 0.02% likelihood
Sample 4: Correct, it was Unknown
  Prediction 1: Unknown with 0.00% likelihood
  Prediction 2: PP with 0.00% likelihood
  Prediction 3: HDPE with 99.70% likelihood
  Pred

In [ ]:

exclude_alu_indices = y != 'ALU'
X_filtered = X[exclude_alu_indices]
y_filtered = y_encoded[exclude_alu_indices]
# Split the filtered data into train and test sets (Filtered ALU)
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_filtered, test_size=0.2, random_state=50)
#features
X_test_normalized = ((X_test - 0) / (8410000 - 0))

# Make predictions
predictions = model.predict(X_test_normalized)
predicted_labels = np.argmax(predictions, axis=1)
predicted_plastic_types = label_encoder.inverse_transform(predicted_labels)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_normalized, y_test)

# Make predictions
predictions = model.predict(X_test_normalized)

# Get the top 4 predictions for each sample
top_predictions = np.argsort(predictions, axis=1)[:, -4:]
top_labels = top_predictions[:, ::-1]

# print(predictions)

# Reshape the top_labels array to be 1D
top_labels_1d = top_labels.reshape(-1)

# Inverse transform the labels
top_plastic_types = label_encoder.inverse_transform(top_labels_1d)

# Reshape the top_plastic_types array back to 2D
top_plastic_types = top_plastic_types.reshape(top_labels.shape)

# Get the percentage likelihoods for the top 4 predictions
percentage_likelihoods = np.take_along_axis(predictions, top_predictions, axis=1) * 100

# Display test accuracy and example predictions
print(f"Test Accuracy: {test_accuracy}")
for i in range(len(predicted_labels)):
    num_predictions = min(4, len(np.unique(top_labels[i])))
    if y_test[i] == predicted_labels[i]:
        CorF = "Correct,"
    else:
        CorF = "False,"
    print(f"Sample {i+1}: {CorF} it was {label_encoder.classes_[y_test[i]]}")
    for j in range(num_predictions):
        print(f"  Prediction {j+1}: {top_plastic_types[i][j]} with {percentage_likelihoods[i][3-j]:.2f}% likelihood")


10/10 [==============================] - 0s 2ms/step
Test Accuracy: 0.9775640964508057
Sample 1: Correct, it was PET
  Prediction 1: PET with 100.00% likelihood
  Prediction 2: PVC with 0.00% likelihood
  Prediction 3: Unknown with 0.00% likelihood
  Prediction 4: PP with 0.00% likelihood
Sample 2: Correct, it was PVC
  Prediction 1: PVC with 99.06% likelihood
  Prediction 2: Unknown with 0.94% likelihood
  Prediction 3: HDPE with 0.00% likelihood
  Prediction 4: PP with 0.00% likelihood
Sample 3: Correct, it was HDPE
  Prediction 1: HDPE with 100.00% likelihood
  Prediction 2: Unknown with 0.00% likelihood
  Prediction 3: PVC with 0.00% likelihood
  Prediction 4: LDPE with 0.00% likelihood
Sample 4: Correct, it was HDPE
  Prediction 1: HDPE with 99.71% likelihood
  Prediction 2: LDPE with 0.29% likelihood
  Prediction 3: Unknown with 0.00% likelihood
  Prediction 4: PVC with 0.00% likelihood
Sample 5: Correct, it was Unknown
  Prediction 1: Unknown with 100.00% likelihood
  Prediction

In [ ]:
# Get manual input from the user
manual_input = np.array([[1271389,2107581,3534859,5988057,6233631,3880377,1498541,985865,1561809,2707983,4719147,6111335,5063453,2701889,1262325]])  # Replace with your input values

# Normalize the manual input if needed
normalized_manual_input = ((manual_input - 0) / (8410000 - 0))

# Make predictions on manual input
predictions = model.predict(normalized_manual_input)
print(predictions)

# Get the top prediction
top_prediction = np.argmax(predictions)
print(top_prediction)

top_plastic_type = label_encoder.inverse_transform([top_prediction])[0]
print(top_plastic_type)
likelihood = predictions[0][top_prediction] * 100

# Display the prediction for manual input
print(f"Manual Input Prediction:")
print(f"  Prediction: {top_plastic_type} with {likelihood:.2f}% likelihood")

1/1 [==============================] - 0s 132ms/step
[[6.2055269e-04 2.0557351e-03 0.0000000e+00 1.8658116e-06 7.5102305e-22
  9.9732190e-01]]
5
Unknown
Manual Input Prediction:
  Prediction: Unknown with 99.73% likelihood


In [ ]:
import numpy as np  # Add this line to import the NumPy library
import serial
from sklearn.preprocessing import LabelEncoder


# Open the serial port
ser = serial.Serial('COM3', baudrate=115200)  # Adjust the baudrate as needed

# Initialize a variable to store the received data
received_data = ""

print(label_encoder.classes_)

try:
    while True:
        # Read a line of data from the serial port (assuming data is sent as lines)
        line = ser.readline().decode('utf-8').strip()
        
        # Append the received line to the variable
        received_data += line

        # Split the received data into a list of numbers using a comma as the delimiter
        data_values = [float(val) for val in received_data.split(',')]

        # Convert the list of values into a NumPy array
        manual_input = np.array([data_values])

        # Normalize the manual input if needed
        normalized_manual_input = ((manual_input - 0) / (8410000 - 0))

        # Make predictions on manual input
        predictions = model.predict(normalized_manual_input)
        formatted_predictions = np.around(predictions, 4)
        print("predictions:")
        print(formatted_predictions)

        # Get the top prediction
        top_prediction = np.argmax(predictions)
        print(top_prediction)

        top_plastic_type = label_encoder.inverse_transform([top_prediction])[0]
        print(top_plastic_type)
        likelihood = predictions[0][top_prediction] * 100

        # Display the prediction for manual input
        print(f"Manual Input Prediction:")
        print(f"  Prediction: {top_plastic_type} with {likelihood:.2f}% likelihood")

        received_data = ""

        # Send the top plastic type back over serial
        ser.write(top_plastic_type.encode('utf-8'))

except KeyboardInterrupt:
    # Handle Ctrl+C to stop the loop gracefully
    pass

finally:
    # Close the serial port when done
    ser.close()

# Now, you can use the 'received_data' variable for further processing
print("total data:\n", received_data)


SerialException: could not open port 'COM3': PermissionError(13, 'Access is denied.', None, 5)

In [ ]:
import numpy as np
import serial
import csv  # Import the csv module
from sklearn.preprocessing import LabelEncoder

# Open the serial port
ser = serial.Serial('COM3', baudrate=115200)  # Adjust the baudrate as needed

# Create and open a CSV file for writing
csv_file = open('adding.csv', 'w', newline='')
csv_writer = csv.writer(csv_file)

try:
    while True:
        # Read a line of data from the serial port (assuming data is sent as lines)
        line = ser.readline().decode('utf-8').strip()

        # Split the received data into a list of numbers using a comma as the delimiter
        data_values = [float(val) for val in line.split(',')]

        print(data_values)

        # Write the data values to the CSV file as a new row
        csv_writer.writerow(data_values)

except KeyboardInterrupt:
    # Handle Ctrl+C to stop the loop gracefully
    pass

finally:
    # Close the CSV file and the serial port when done
    csv_file.close()
    ser.close()

# The received data has been saved in the 'received_data.csv' file
print("Data saved to 'PP2.csv'")



[1406775.0, 2438191.0, 4226915.0, 8352103.0, 7489279.0, 4897563.0, 2166151.0, 1371501.0, 1825375.0, 3242003.0, 6387449.0, 8089319.0, 6254421.0, 3567105.0, 1774555.0]
[1393103.0, 2432703.0, 4190545.0, 8352151.0, 7446563.0, 4889489.0, 2160419.0, 1383655.0, 1817711.0, 3222841.0, 6335937.0, 8036565.0, 6252681.0, 3568443.0, 1748437.0]
[1401341.0, 2440199.0, 4264819.0, 8352331.0, 7444527.0, 4876063.0, 2131495.0, 1380653.0, 1809193.0, 3232141.0, 6383169.0, 8053585.0, 6215115.0, 3552345.0, 1769345.0]
[1377053.0, 2410033.0, 4172373.0, 8354063.0, 7394961.0, 4883509.0, 2127933.0, 1370023.0, 1798435.0, 3194839.0, 6322179.0, 8006141.0, 6180733.0, 3525669.0, 1756779.0]
[1390663.0, 2412993.0, 4153111.0, 8354601.0, 7439031.0, 4881111.0, 2107891.0, 1360875.0, 1800815.0, 3195079.0, 6313829.0, 8016551.0, 6183093.0, 3509581.0, 1748849.0]
[848573.0, 1544555.0, 2643857.0, 5520829.0, 4858181.0, 3562901.0, 1605133.0, 949353.0, 1110323.0, 1973011.0, 3959749.0, 5161323.0, 4234283.0, 2601743.0, 1281189.0]
[11328

SerialException: GetOverlappedResult failed (PermissionError(13, 'Access is denied.', None, 5))